In [ ]:
import mysql.connector
from datetime import datetime, timedelta
import schedule
import time

def preprocess_and_display():
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="IOT"
    )
    cursor = conn.cursor()

    cursor.execute("SELECT DISTINCT node_id FROM NodeSensor")
    id_nodes = cursor.fetchall()

    for id_node in id_nodes:
        id_node = id_node[0]

        cursor.execute("SELECT MAX(id) FROM NodeSensor WHERE node_id = %s", (id_node,))
        last_node_sensor_id = cursor.fetchone()[0]

        if last_node_sensor_id is not None:
            cursor.execute("SELECT * FROM NodeSensor WHERE id = %s", (last_node_sensor_id,))
            last_node_sensor_data = cursor.fetchone()

            if last_node_sensor_data is not None:
                node_sensor_id = last_node_sensor_data[0]
                kondisi_battery = last_node_sensor_data[3]

                cursor.execute("""
                    SELECT data_kelembapan_tanah_1, data_kelembapan_tanah_2
                    FROM NodeSensor
                    WHERE id = %s
                """, (last_node_sensor_id,))
                kelembapan_avg = cursor.fetchone()
                akumulasi_kelembapan = int((kelembapan_avg[0] + kelembapan_avg[1]) / 2)

                cursor.execute("""
                    SELECT data_kemiringan_x, data_kemiringan_y, data_kemiringan_z
                    FROM NodeSensor
                    WHERE node_id = %s
                    ORDER BY id DESC
                    LIMIT 3
                """, (id_node,))
                kemiringan_data = cursor.fetchall()

                if kemiringan_data and all(item is not None for item in kemiringan_data[0]):
                    x, y, z = kemiringan_data[0]
                    initial_kemiringan_data = (0, 0, 0)
                    accumulated_percentage_change = calculate_accumulated_percentage_change(initial_kemiringan_data, (x, y, z))
                else:
                    accumulated_percentage_change = 0

                status_waspada = determine_alert_status(accumulated_percentage_change, last_node_sensor_data[4], akumulasi_kelembapan)

                current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

                print(f"Node ID: {id_node}")
                print(f"Node Sensor ID: {node_sensor_id}")
                print(f"Kondisi Battery: {kondisi_battery}")
                print(f"Akumulasi Kelembapan: {akumulasi_kelembapan}%")
                print(f"Akumulasi Perubahan Kemiringan: {accumulated_percentage_change}%")
                print(f"Status Waspada: {status_waspada}")
                print(f"Timestamp: {current_time}")
                print()

                # Menyimpan data ke database
                insert_data_to_database(id_node, node_sensor_id, kondisi_battery, akumulasi_kelembapan, accumulated_percentage_change, status_waspada, current_time)

    conn.close()

def calculate_accumulated_percentage_change(initial_data, current_data):
    delta_x = current_data[0] - initial_data[0]
    delta_y = current_data[1] - initial_data[1]
    delta_z = current_data[2] - initial_data[2]

    max_tilt = 19
    percentage_change_x = (delta_x / max_tilt) * 100
    percentage_change_y = (delta_y / max_tilt) * 100
    percentage_change_z = (delta_z / max_tilt) * 100

    accumulated_percentage_change = int((percentage_change_x + percentage_change_y + percentage_change_z) / 3)

    return accumulated_percentage_change

def determine_alert_status(accumulated_percentage_change, status_hujan, akumulasi_kelembapan):
    if accumulated_percentage_change > 0.5 and status_hujan:
        return "Bahaya"
    elif accumulated_percentage_change > 0.2 or akumulasi_kelembapan > 40:
        return "Waspada"
    else:
        return "Normal"

def insert_data_to_database(id_node, node_sensor_id, kondisi_battery, akumulasi_kelembapan, accumulated_percentage_change, status_waspada, current_time):
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="IOT"
    )
    cursor = conn.cursor()

    if status_waspada == "Normal":
        status_waspada_value = 0
    elif status_waspada == "Waspada":
        status_waspada_value = 1
    elif status_waspada == "Bahaya":
        status_waspada_value = 3
    else:
        status_waspada_value = None

    insert_query = """
        INSERT INTO PreprocessingLog (id_node, NodeSensor_id, kondisi_battrey, status_hujan, akumulasi_kelembapan,
        perubahan_kemiringan, perubahan_lokasi, status_waspada, timestamp)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    cursor.execute(insert_query, (
        id_node, node_sensor_id, kondisi_battery, 0, akumulasi_kelembapan,
        accumulated_percentage_change, 0, status_waspada_value, current_time
    ))

    conn.commit()
    conn.close()

def job():
    preprocess_and_display()

schedule.every(10).seconds.do(job)

while True:
    schedule.run_pending()
    time.sleep(1)


Node ID: 3844
Node Sensor ID: 1026
Kondisi Battery: 1
Akumulasi Kelembapan: 23%
Akumulasi Perubahan Kemiringan: 15%
Status Waspada: Bahaya
Timestamp: 2023-12-10 18:35:57

Node ID: 3844
Node Sensor ID: 1026
Kondisi Battery: 1
Akumulasi Kelembapan: 23%
Akumulasi Perubahan Kemiringan: 15%
Status Waspada: Bahaya
Timestamp: 2023-12-10 18:35:57

Node ID: 3844
Node Sensor ID: 1026
Kondisi Battery: 1
Akumulasi Kelembapan: 23%
Akumulasi Perubahan Kemiringan: 15%
Status Waspada: Bahaya
Timestamp: 2023-12-10 18:35:57

Node ID: 3844
Node Sensor ID: 1026
Kondisi Battery: 1
Akumulasi Kelembapan: 23%
Akumulasi Perubahan Kemiringan: 15%
Status Waspada: Bahaya
Timestamp: 2023-12-10 18:36:07

Node ID: 3844
Node Sensor ID: 1026
Kondisi Battery: 1
Akumulasi Kelembapan: 23%
Akumulasi Perubahan Kemiringan: 15%
Status Waspada: Bahaya
Timestamp: 2023-12-10 18:36:07

Node ID: 3844
Node Sensor ID: 1026
Kondisi Battery: 1
Akumulasi Kelembapan: 23%
Akumulasi Perubahan Kemiringan: 15%
Status Waspada: Bahaya
Timest